In [114]:
import pandas as pd
import geopandas as gpd
import numpy as np
import csv 

The Census Bureau provides 'relationship files' that show how to convert 2010 and 2020 fips codes.  
https://www.census.gov/geographies/reference-files/time-series/geo/relationship-files.2020.html#tract

In the cell below, I used chatGPT to help me write function that would convert the Census Bureau's .txt file to a csv file

In [10]:

# Define the path to your input text file
text_file_path = '/Users/jacoblapp/Downloads/tab20_tract20_tract10_natl.txt'

# Define the path to the output CSV file
csv_file_path = '/Users/jacoblapp/Downloads/tract_output.csv'

# Define the custom delimiter for the text file
custom_delimiter = '|'  # this is the delimiter used in the Census Bureau's relationship file

# Open the input text file in read mode
with open(text_file_path, 'r') as text_file:
    # Read the lines of the text file
    lines = text_file.readlines()

# Open the output CSV file in write mode
with open(csv_file_path, 'w', newline='') as csv_file:
    # Create a CSV writer object
    csv_writer = csv.writer(csv_file)

    # Iterate through each line in the text file
    for line in lines:
        # Remove leading and trailing whitespace
        line = line.strip()

        # Split the line into fields using the custom delimiter
        fields = line.split(custom_delimiter)

        # Write the fields to the CSV file
        csv_writer.writerow(fields)

print(f'Text file "{text_file_path}" converted to CSV file "{csv_file_path}" successfully.')


Text file "/Users/jacoblapp/Downloads/tab20_tract20_tract10_natl.txt" converted to CSV file "/Users/jacoblapp/Downloads/tract_output.csv" successfully.


In [58]:
tract_sheet = pd.read_csv('/Users/jacoblapp/Downloads/tract_output.csv', skipinitialspace=True)
tract_sheet.head()

,OID_TRACT_20,GEOID_TRACT_20,NAMELSAD_TRACT_20,AREALAND_TRACT_20,AREAWATER_TRACT_20,MTFCC_TRACT_20,FUNCSTAT_TRACT_20,OID_TRACT_10,GEOID_TRACT_10,NAMELSAD_TRACT_10,AREALAND_TRACT_10,AREAWATER_TRACT_10,MTFCC_TRACT_10,FUNCSTAT_TRACT_10,AREALAND_PART,AREAWATER_PART
0,20790540092527,1001020100,Census Tract 201,9825304,28435,G5020,S,20740540092527,1001020100,Census Tract 201,9827271,28435,G5020,S,9820448,28435
1,20790540092527,1001020100,Census Tract 201,9825304,28435,G5020,S,20740540092534,1001020200,Census Tract 202,3325674,5669,G5020,S,4856,0
2,20790540092534,1001020200,Census Tract 202,3320818,5669,G5020,S,20740540092534,1001020200,Census Tract 202,3325674,5669,G5020,S,3320818,5669
3,20790540092528,1001020300,Census Tract 203,5349271,9054,G5020,S,20740540092528,1001020300,Census Tract 203,5349271,9054,G5020,S,5349271,9054
4,20790540092529,1001020400,Census Tract 204,6384282,8408,G5020,S,20740540092529,1001020400,Census Tract 204,6384282,8408,G5020,S,6384282,8408


Now I load the ejscreen geopandas database as 'ej' and I load the cejst data as 'cejst'

In [13]:
ej = gpd.read_file('/Users/jacoblapp/Desktop/Sci4GA/sci4ga_ej_tools_and_data/EJSCREEN_2023_Tracts_StatePct_with_AS_CNMI_GU_VI.gdb')
ga_ej = ej[ej['STATE_NAME'] == 'Georgia']

cejst = gpd.read_file('/Users/jacoblapp/Desktop/Sci4GA/sci4ga_ej_tools_and_data/cejst.shp')
ga_cejst = cejst[cejst['SF'] == 'Georgia']

In [22]:
tract_sheet.loc[:, ['GEOID_TRACT_20', 'GEOID_TRACT_10']]

,GEOID_TRACT_20,GEOID_TRACT_10
0,1001020100,1001020100
1,1001020100,1001020200
2,1001020200,1001020200
3,1001020300,1001020300
4,1001020400,1001020400
...,...,...
126445,78030961100,78030961200
126446,78030961200,78030960400
126447,78030961200,78030961100
126448,78030961200,78030961200


In [122]:
ej_ids = np.int64(pd.Series(ga_ej.ID))
cejst_ids = np.int64(pd.Series(ga_cejst.GEOID10))
id20 = pd.Series(tract_sheet.GEOID_TRACT_20)
id10 = pd.Series(tract_sheet.GEOID_TRACT_10)

In [123]:
print(len(cejst_ids), len(ej_ids), len(id20), len(id10))

1969 2796 126450 126450


In [189]:
print(any(element in ej_ids for element in id20))
print(ej_ids.dtype == cejst_ids.dtype == id20.dtype == id10.dtype) # need all same data types
print(id20.dtype, id10.dtype, cejst_ids.dtype, ej_ids.dtype) # gotta convert our series from shp files to int64

True
True
int64 int64 int64 int64


Now that we have all the same correct data types, we need to subset the tracts relationship dataframe.

We need to only look at the rows of the tracts dataframe that have the ids from CEJST. 
Once we find these rows, we should be able to find the 2020 codes

In [167]:
old_ids = pd.Series(cejst_ids[np.isin(cejst_ids, tract_sheet['GEOID_TRACT_10'])])
newdf = tract_sheet[np.isin(tract_sheet['GEOID_TRACT_10'], old_ids)]

In [175]:
print(len(newdf.GEOID_TRACT_20.unique()), len(ej_ids), len(newdf.GEOID_TRACT_10.unique()))
print('Return to this, it seems like we may have added 3 new fips codes. BUT, I believe this process is correct. in the next cell i figure it out.')

2799 2796 1969
Return to this, it seems like we may have added 3 new fips codes. BUT, I believe this process is correct. in the next cell i figure it out.


In [181]:
ga_fips_conv = newdf.loc[:, ['GEOID_TRACT_20', 'GEOID_TRACT_10']]
print(ga_fips_conv)
print('the last three rows are nonsense, this explains earlier.')

       GEOID_TRACT_20  GEOID_TRACT_10
30974     13001950100     13001950100
30975     13001950201     13001950200
30976     13001950202     13001950200
30977     13001950301     13001950300
30978     13001950302     13001950300
...               ...             ...
34691     13321950500     13321950500
34692     13321950600     13321950600
82048     37039930604     13111050100
99906     45053950302     13051010605
99909     45053990100     13051990000

[3714 rows x 2 columns]
the last three rows are nonsense, this explains earlier.


In [183]:
ga_fips_conv = ga_fips_conv[:-3]
print(ga_fips_conv)
print('Fixed')

       GEOID_TRACT_20  GEOID_TRACT_10
30974     13001950100     13001950100
30975     13001950201     13001950200
30976     13001950202     13001950200
30977     13001950301     13001950300
30978     13001950302     13001950300
...               ...             ...
34685     13319960300     13319960300
34686     13319960400     13319960400
34687     13321950100     13321950100
34688     13321950201     13321950200
34689     13321950202     13321950200

[3708 rows x 2 columns]
Fixed


Writing new conversion dataframe to csv.

In [188]:
ga_fips_conv = pd.DataFrame(ga_fips_conv)
file_path = '/Users/jacoblapp/Desktop/Sci4GA/sci4ga_ej_tools_and_data/ga_fips_conv.csv'
ga_fips_conv.to_csv(file_path, index = False)
print('converted to csv')

converted to csv
